In [1]:
# import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')
# !pip install textblob
# !pip install gensim
# !python -m spacy download en_core_web_sm
# !pip install transformers
# !pip install sentencepiece
# !pip install datsets transformers[sentencepiece]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2023-11-15 14:15:40.995090: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 14:15:40.995155: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 14:15:40.995200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 14:15:41.006638: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 14:15:43.397364: W tensorflow/compiler/

Using Flan T5 Model without finetuning it

In [4]:
import re
import numpy as np
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords

folder_path = "/content/"

train_df = pd.read_csv(folder_path+"NLP_ass_train.tsv", delimiter="\t")
test_df = pd.read_csv(folder_path+"NLP_ass_test.tsv", delimiter = "\t")
val_df = pd.read_csv(folder_path+"NLP_ass_valid.tsv", delimiter = "\t")

# Renaming the columns
df1 = pd.DataFrame({train_df.columns[0]: [train_df.columns[0]], train_df.columns[1]: [train_df.columns[1]]})
train_df = pd.concat([df1, train_df])
train_df = train_df.reset_index(drop=True)
train_df.columns = ["Text", "Label"]

df2 = pd.DataFrame({val_df.columns[0]: [val_df.columns[0]], val_df.columns[1]: [val_df.columns[1]]})
val_df = pd.concat([df2, val_df])
val_df = val_df.reset_index(drop=True)
val_df.columns = ["Text", "Label"]

df3 = pd.DataFrame({test_df.columns[0]: [test_df.columns[0]], test_df.columns[1]: [test_df.columns[1]]})
test_df = pd.concat([df3, test_df])
test_df = test_df.reset_index(drop=True)
test_df.columns = ["Text", "Label"]

In [5]:
def preprocess_prompt(prompt):
    # Convert prompt to lowercase
    prompt = prompt.lower()
    # Remove special characters and symbols
    prompt = re.sub(r'[^a-zA-Z0-9\s]', '', prompt)
    return prompt

In [6]:
# Preprocessing the data
train_df['Text'] = train_df['Text'].apply(preprocess_prompt)
val_df['Text'] = val_df['Text'].apply(preprocess_prompt)
test_df['Text'] = test_df['Text'].apply(preprocess_prompt)

In [7]:
train_prompts = train_df["Text"].to_list()
test_prompts = test_df["Text"].to_list()
val_prompts = val_df["Text"].to_list()

In [9]:
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer

# Download Flan-T5 small model
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer

# Download Flan-T5 base model
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [10]:
def zero_shot_hate_speech_classification(text):
    zero_shot_prompt = """Task: Text Classification
    Definitions:
    1. hatespeech: Any speech or text that specifically targets and attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, gender, disability, sexual orientation, or gender identity.
    2. offensive: Text or speech containing abusive slurs or derogatory terms, which may not necessarily target a specific attribute but is disrespectful or inappropriate.
    3. normal: Text that is neutral, respectful, and adheres to social norms.
    Your task is to classify the following text snippet, enclosed within three backticks, as either normal, offensive, or hatespeech based on the given definitions. Please provide the classification as either normal, offensive, or hatespeech. Be precise in your classification according to the provided definitions.
    Text to classify: """ + f'{text}'
    task_description = zero_shot_prompt
    prompt = task_description.format(text)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output

In [11]:
test_df['zero shot predictions'] = test_df['Text'].apply(zero_shot_hate_speech_classification)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [14]:
test_df

,Text,Label,zero shot predictions
0,i dont think im getting my baby them white 9 h...,normal,normal
1,laura loomer raped me while screaming at me in...,hatespeech,hatespeech
2,user what did the old lady do was she a nazi o...,normal,normal
3,as much as i appreciate a format of open discu...,normal,normal
4,sex be so good a bitch be slow stroking and cr...,offensive,normal
...,...,...,...
1919,it always women trying this shit like dingbat ...,offensive,hatespeech
1920,because women would never lie about being sexu...,offensive,normal
1921,how is all that awesome muslim diversity going...,offensive,normal
1922,well my dear lgbtq brothers and sisters i do n...,hatespeech,hatespeech


In [15]:
import pandas as pd

# Calculate accuracy directly without creating a new column
accuracy = (test_df['Label'] == test_df['zero shot predictions']).mean()
print("\nOverall Accuracy:", accuracy)

# Accuracy = 49.79209979209979 %


Overall Accuracy: 0.4979209979209979


In [30]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

# Encode labels to numeric values
le = LabelEncoder()
test_df['zero shot true_labels_encoded'] = le.fit_transform(test_df['Label'])
test_df['zero shot predicted_labels_encoded'] = le.transform(test_df['zero shot predictions'])

# Calculate macro F1 score
macro_f1 = f1_score(test_df['zero shot true_labels_encoded'], test_df['zero shot predicted_labels_encoded'], average='macro')

# Display the macro F1 score
print("Macro F1 Score:", macro_f1)

# Macro F1 Score: 0.3854591244009152

Macro F1 Score: 0.3854591244009152


In [21]:
def few_shot_hate_speech_classification(text):
    few_shot_prompt = """Task: Text Classification
    Definitions:
    1. hatespeech: Any speech or text that specifically targets and attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, gender, disability, sexual orientation, or gender identity.
    2. offensive: Text or speech containing abusive slurs or derogatory terms, which may not necessarily target a specific attribute but is disrespectful or inappropriate.
    3. normal: Text that is neutral, respectful, and adheres to social norms.
    Your task is to classify the following text snippet, enclosed within three backticks, as either normal, offensive, or hatespeech based on the given definitions. Please provide the classification as either normal, offensive, or hatespeech. Be precise in your classification according to the provided definitions.
    Examples:
    1) Input: we cannot continue calling ourselves feminists if the rights of all womxn arent addressed yes to a sexual offences public list but will a trans lesbian bisexual and queer womxn be able to enter their information on the reporting sheet gender forum
    Output: normal

    2) Input: i saw a couple baums and steins in the victim list but that about it ya got more info i thought the school was half spic though
    Output: offensive

    3) Input: guilty until proven innocent unless you are a jew a nigger or a lover of both
    Output: hatespeech.

    Text to classify: """ + f'{text}'

    task_description = few_shot_prompt
    prompt = task_description.format(text)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output

In [24]:
test_df['few shot predictions'] = test_df['Text'].apply(few_shot_hate_speech_classification)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [27]:
test_df['few shot predictions'] = test_df['few shot predictions'].str.replace('Output: ', '')
test_df

,Text,Label,zero shot predictions,few shot predictions
0,i dont think im getting my baby them white 9 h...,normal,normal,normal
1,laura loomer raped me while screaming at me in...,hatespeech,hatespeech,hatespeech
2,user what did the old lady do was she a nazi o...,normal,normal,hatespeech
3,as much as i appreciate a format of open discu...,normal,normal,normal
4,sex be so good a bitch be slow stroking and cr...,offensive,normal,offensive
...,...,...,...,...
1919,it always women trying this shit like dingbat ...,offensive,hatespeech,hatespeech
1920,because women would never lie about being sexu...,offensive,normal,normal
1921,how is all that awesome muslim diversity going...,offensive,normal,normal
1922,well my dear lgbtq brothers and sisters i do n...,hatespeech,hatespeech,normal


In [28]:
import pandas as pd

# Calculate accuracy directly without creating a new column
accuracy = (test_df['Label'] == test_df['few shot predictions']).mean()
print("\nOverall Accuracy:", accuracy)

# Accuracy = 50.36382536382537 %


Overall Accuracy: 0.5036382536382537


In [31]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

# Encode labels to numeric values
le = LabelEncoder()
test_df['few shot true_labels_encoded'] = le.fit_transform(test_df['Label'])
test_df['few shot predicted_labels_encoded'] = le.transform(test_df['few shot predictions'])

# Calculate macro F1 score
macro_f1 = f1_score(test_df['few shot true_labels_encoded'], test_df['few shot predicted_labels_encoded'], average='macro')

# Display the macro F1 score
print("Macro F1 Score:", macro_f1)

# Macro F1 Score: 0.41416884961279377

Macro F1 Score: 0.41416884961279377


In [32]:
def calculate_intersection(test_set, train_set, validation_set):
    intersection_count = len(set(test_set) & set(train_set) & set(validation_set))
    return intersection_count

intersection_result = calculate_intersection(test_df, train_df, val_df)
print(f"Intersection between test set and train/validation sets: {intersection_result}")

Intersection between test set and train/validation sets: 2
